In [1]:
#Importing the essential libraries
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
#Loading the dataset
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Projects\Python\Language Detection Model\Language Detection.csv")
df

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
10332,ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...,Kannada
10333,ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...,Kannada
10334,ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎ...,Kannada
10335,ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...,Kannada


In [3]:
#Loading English raw data from the dataset
english_df = df[df['Language'] == 'English'].rename(columns={'Text': 'English'})
english_df

,English,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
1380,what your mistake was come let's feed you some...,English
1381,narcisa changed her ways she struggled at firs...,English
1382,how's narcissism now marian told them both eve...,English
1383,has she i guess she wouldn't want any more gol...,English


In [4]:
#Loading German raw data from the dataset
german_df = df[df['Language'] == 'German'].rename(columns={'Text': 'German'})
german_df

,German,Language
9498,.Wir sind alle auf der Suche nach schnellen We...,German
9499,"Ich weiß, ich verstehe, was passiert, ist, das...",German
9500,Wie wäre es also mit uns?,German
9501,Lernen Sie einige intelligente neue englische ...,German
9502,Also werden wir heute 10 neue intelligente Wör...,German
...,...,...
9963,narcisa änderte ihre art und weise sie kämpfte...,German
9964,Wie' s Narzissmus jetzt erzählt Marian beiden ...,German
9965,"Hat sie, ich denke, sie würde jetzt kein Goldb...",German
9966,"Terry, du siehst tatsächlich ein bisschen wie ...",German


In [5]:
#Loading French raw data from the dataset
french_df = df[df['Language'] == 'French'].rename(columns={'Text': 'French'})
french_df

,French,Language
3250,Si vous disposez d'ouvrages ou d'articles de r...,French
3251,Comment ajouter mes sources ?,French
3252,Cette page ou section est en train d'être trad...,French
3253,Vous pouvez aider au développement de Wikipédi...,French
3254,Le mot nature est un terme polysémique (c’est-...,French
...,...,...
4259,"quelle a été votre erreur, nous allons vous no...",French
4260,"narcisa a changé ses manières, elle a lutté au...",French
4261,Comment' Le narcissisme de s maintenant marian...,French
4262,a-t-elle je suppose qu'elle ne voudrait plus d...,French


In [6]:
#Loading Spanish raw data from the dataset
spanish_df = df[df['Language'] == 'Spanish'].rename(columns={'Text': 'Spanish'})
spanish_df

,Spanish,Language
4810,Nature es una de las más prestigiosas revistas...,Spanish
4811,Su primer número fue publicado el 4 de noviemb...,Spanish
4812,"[2]​ Con una periodicidad semanal, la revista ...",Spanish
4813,"Nature tiene oficinas en Londres, Nueva York, ...",Spanish
4814,La editorial publica también varias revistas e...,Spanish
...,...,...
5624,¿Tiene ella?,Spanish
5625,Supongo que no querría más pan de oro ahora ¿e...,Spanish
5626,"Terry, en realidad te pareces un poco a ese án...",Spanish
5627,¿Cómo pudiste ser él?,Spanish


In [7]:
#Loading Malayalam raw data from the dataset
malayalam_df = df[df['Language'] == 'Malayalam'].rename(columns={'Text': 'Malayalam'})
malayalam_df

,Malayalam,Language
1385,ഭൗതികപ്രപഞ്ചത്തെ മൊത്തത്തിൽ സൂചിപ്പിക്കുന്ന പദ...,Malayalam
1386,ഭൗതികപ്രതിഭാസങ്ങളും ജീവനും പ്രകൃതിയുടെ ഘടകങ്ങള...,Malayalam
1387,മനുഷ്യനിർമിതമായ വസ്തുക്കളെ പ്രകൃതിയുടെ ഭാഗമായി...,Malayalam
1388,അവയെ കൃത്രിമം എന്ന് വിശേഷിപ്പിക്കുന്നുഅഭിപ്രായ...,Malayalam
1389,പ്രകൃതി എന്ന പദം പ്രപഞ്ചത്തെയും അതിലെ സമസ്ത പ്...,Malayalam
...,...,...
1974,നിങ്ങളുടെ തെറ്റ് വന്നത് ആ ദിവസം മുതൽ നിങ്ങൾക്ക...,Malayalam
1975,"നാർസിസ ആദ്യം പാടുപെട്ട വഴികൾ മാറ്റി, പക്ഷേ പതു...",Malayalam
1976,എങ്ങനെ' സംഭവിച്ച എല്ലാ കാര്യങ്ങളും ഇപ്പോൾ മരിയ...,Malayalam
1977,"അവൾക്കറിയാമോ, അവൾക്ക് ഇപ്പോൾ കൂടുതൽ സ്വർണ്ണ റൊ...",Malayalam


In [8]:
#Identifying the punctuations
for char in string.punctuation:
    print(char,end=" ")
translate_table=dict((ord(char),None) for char in string.punctuation)

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [9]:
#Pre-processing Engish Raw data
data_english = []
lang_english=[]
for i,line in english_df.iterrows():
    line=line['English']
    if len(line) !=0:
        line=line.lower()
        line=re.sub(r"\d+","",line)
        line=line.translate(translate_table)
        data_english.append(line)
        lang_english.append("English")

In [10]:
#Pre-processing German Raw data
data_german = []
lang_german=[]
for i,line in german_df.iterrows():
    line=line['German']
    if len(line) !=0:
        line=line.lower()
        line=re.sub(r"\d+","",line)
        line=line.translate(translate_table)
        data_german.append(line)
        lang_german.append("German")

In [11]:
#Pre-processing French Raw data
data_french = []
lang_french=[]
for i,line in french_df.iterrows():
    line=line['French']
    if len(line) !=0:
        line=line.lower()
        line=re.sub(r"\d+","",line)
        line=line.translate(translate_table)
        data_french.append(line)
        lang_french.append("French")

In [12]:
#Pre-processing Spanish Raw data
data_spanish = []
lang_spanish=[]
for i,line in spanish_df.iterrows():
    line=line['Spanish']
    if len(line) !=0:
        line=line.lower()
        line=re.sub(r"\d+","",line)
        line=line.translate(translate_table)
        data_spanish.append(line)
        lang_spanish.append("Spanish")

In [13]:
#Pre-processing Malayalam Raw data
data_malayalam = []
lang_malayalam=[]
for i,line in malayalam_df.iterrows():
    line=line['Malayalam']
    if len(line) !=0:
        line=line.lower()
        line=re.sub(r"\d+","",line)
        line=re.sub(r"[a-zA-Z]","",line)
        line=line.translate(translate_table)
        data_malayalam.append(line)
        lang_malayalam.append("Malayalam")

In [14]:
#Visualizing the data
df=pd.DataFrame({"Text":data_english+data_german+data_french+data_spanish+data_malayalam,"Language":lang_english+lang_german+lang_french+lang_spanish+lang_malayalam})
df

,Text,Language
0,nature in the broadest sense is the natural p...,English
1,nature can refer to the phenomena of the physi...,English
2,the study of nature is a large if not the only...,English
3,although humans are part of nature human activ...,English
4,the word nature is borrowed from the old fren...,English
...,...,...
4277,നിങ്ങളുടെ തെറ്റ് വന്നത് ആ ദിവസം മുതൽ നിങ്ങൾക്ക...,Malayalam
4278,നാർസിസ ആദ്യം പാടുപെട്ട വഴികൾ മാറ്റി പക്ഷേ പതുക...,Malayalam
4279,എങ്ങനെ സംഭവിച്ച എല്ലാ കാര്യങ്ങളും ഇപ്പോൾ മരിയൻ...,Malayalam
4280,അവൾക്കറിയാമോ അവൾക്ക് ഇപ്പോൾ കൂടുതൽ സ്വർണ്ണ റൊട...,Malayalam


In [15]:
#Splitting the data into train and test sets
x,y = df.iloc[:,0],df.iloc[:,1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=20)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2997,)
(1285,)
(2997,)
(1285,)


In [16]:
#Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char')
pipe_lr_r13 = pipeline.Pipeline([('vectorizer', vectorizer), ('clf', LogisticRegression())])

In [17]:
#Model fitting
pipe_lr_r13.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [18]:
#Model prediction
y_predicted=pipe_lr_r13.predict(x_test)
y_predicted

array(['Spanish', 'English', 'French', ..., 'English', 'English',
       'English'], dtype=object)

In [19]:
#Evaluating the model
acc=(metrics.accuracy_score(y_test,y_predicted))*100
print(acc,"%")

99.29961089494164 %


In [20]:
#Evaluating the model
matrix=(metrics.confusion_matrix(y_test,y_predicted))
print('Confusion Matrix: \n',matrix)

Confusion Matrix: 
 [[441   0   0   0   0]
 [  3 303   0   0   1]
 [  0   1 142   0   0]
 [  0   0   0 187   0]
 [  3   1   0   0 203]]


In [21]:
#Model saving
import pickle
lrFile=open('LRmodel.pckl','wb')
pickle.dump(pipe_lr_r13,lrFile)
lrFile.close()

In [22]:
#Model loading
global lrLangDetectModel
lrLangDetectFile=open('LRmodel.pckl','rb')
lrLangDetectModel=pickle.load(lrLangDetectFile)
lrLangDetectFile.close()

In [23]:
#Calling trained data to make decisions
def lang_detect(text):
    import numpy as np
    import string
    import re
    import pickle
    translate_table=dict((ord(char),None) for char in string.punctuation)
    global lrLangDetectModel
    lrLangDetectFile=open('LRmodel.pckl','rb')
    lrLangDetectModel=pickle.load(lrLangDetectFile)
    lrLangDetectFile.close()
    text=" ".join(text.split())
    text=text.lower()
    text=re.sub(r"\d+","",text)
    text=text.translate(translate_table)
    pred=lrLangDetectModel.predict([text])
    prob=lrLangDetectModel.predict_proba([text])
    return pred[0]

In [24]:
lang_detect("Hello I just build my own language detection model")

'English'

In [25]:
lang_detect("Hallo, ich habe gerade mein eigenes Spracherkennungsmodell erstellt")

'German'

In [26]:
lang_detect("Bonjour, je viens de construire mon propre modèle de détection de langue")

'French'

In [27]:
lang_detect("Hola, acabo de crear mi propio modelo de detección de idioma")

'Spanish'

In [28]:
lang_detect("ഹലോ ഞാൻ എന്റെ സ്വന്തം ഭാഷാ കണ്ടെത്തൽ മോഡൽ നിർമ്മിക്കുന്നു")

'Malayalam'

In [32]:
#Creating GUI for the model
import PySimpleGUI as sg

layout = [
    [sg.Text("Enter Text:")],
    [sg.InputText(key="input_text")],
    [sg.Button("Detect Language"), sg.Button("Exit")],
    [sg.Text(size=(30, 1), key="output_text")]
]

window = sg.Window("Language Detection", layout)

def lang_detect(text):
    import numpy as np
    import string
    import re
    import pickle
    translate_table=dict((ord(char),None) for char in string.punctuation)
    global lrLangDetectModel
    lrLangDetectFile=open('LRmodel.pckl','rb')
    lrLangDetectModel=pickle.load(lrLangDetectFile)
    lrLangDetectFile.close()
    text=" ".join(text.split())
    text=text.lower()
    text=re.sub(r"\d+","",text)
    text=text.translate(translate_table)
    pred=lrLangDetectModel.predict([text])
    prob=lrLangDetectModel.predict_proba([text])
    return pred[0]

while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == "Exit":
        break
    elif event == "Detect Language":
        input_text = values["input_text"]
        if input_text:
            prediction = lang_detect(input_text)
            window["output_text"].update(f"Language: {prediction}")

window.close()


Note: you may need to restart the kernel to use updated packages.
